In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import requests
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime

# Define your API key
api_key = '17b993e3-1e66-445c-989b-7b6f6f12c046'

def get_current_btc_price(api_key):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
    parameters = {
        'symbol': 'BTC',
        'convert': 'USD'
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    response = requests.get(url, headers=headers, params=parameters)
    data = response.json()
    btc_price = data['data']['BTC']['quote']['USD']['price']
    return btc_price

def fetch_historical_data(api_key, symbol='BTC', start_date='2023-01-01', end_date=datetime.now().strftime('%Y-%m-%d')):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/historical'
    parameters = {
        'symbol': symbol,
        'time_start': start_date,
        'time_end': end_date,
    }
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    
    response = requests.get(url, headers=headers, params=parameters)
    
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")
    
    data = response.json()
    
    # Debug: print the keys of the response to see if 'data' is present
    print(data.keys())
    
    # Check if 'data' key is in the response
    if 'data' not in data:
        raise KeyError(f"'data' not found in response. Keys present: {list(data.keys())}")
    
    df = pd.DataFrame(data['data']['quotes'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['close'] = [quote['quote']['USD']['price'] for quote in df['quotes']]
    df.set_index('timestamp', inplace=True)
    df.drop(columns='quotes', inplace=True)
    return df

def compute_rsi(data, window=14):
    delta = data.diff(1).dropna()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Fetch the historical data
# historical_data = fetch_historical_data(api_key, start_date='2023-01-01', end_date=datetime.now().strftime('%Y-%m-%d'))
file_path = 'historical_data.csv'
historical_data = pd.read_csv(file_path)
print(historical_data.head())

# Calculate RSI and Moving Average
historical_data['RSI'] = compute_rsi(historical_data['close'])
historical_data['Moving_Average'] = historical_data['close'].rolling(window=50).mean()
historical_data.dropna(inplace=True)

# Define a target variable for machine learning
historical_data['Future_Price'] = historical_data['close'].shift(-1)
historical_data.dropna(inplace=True)

# Split data into features and target for machine learning
X = historical_data[['close', 'volume', 'Moving_Average', 'RSI']]
y = historical_data['Future_Price']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Model prediction
predictions = model.predict(X_test)

# Evaluate model (implement an evaluation metric)
print("Model Accuracy: ", model.score(X_test, y_test))

# Strategy implementation (simplified example)
def should_sell(current_price, ath_price, threshold=0.05):
    return current_price < ath_price * (1 - threshold)

# Example usage
current_price = get_current_btc_price(api_key)
ath_price = historical_data['close'].max()

if should_sell(current_price, ath_price):
    print("It's time to sell")
else:
    print("Hold")

# Set plot style
plt.style.use('dark_background')

# Plotting Price and Moving Average
plt.figure(figsize=(12, 6))
plt.plot(historical_data.index, historical_data['close'], label='BTC Close Price', alpha=0.5)
plt.plot(historical_data.index, historical_data['Moving_Average'], label='50-Day Moving Average', alpha=0.7)
plt.title('BTC Historical Prices with Moving Average')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
# Format the x-axis to show dates properly
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-rotate dates
plt.show()

# Plotting RSI
plt.figure(figsize=(12, 4))
plt.plot(historical_data.index, historical_data['RSI'], label='RSI', color='purple')
plt.axhline(70, color='red', linestyle='--', alpha=0.5)  # Overbought line
plt.axhline(30, color='green', linestyle='--', alpha=0.5)  # Oversold line
plt.title('Relative Strength Index (RSI)')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.legend()
# Format the x-axis to show dates properly
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-rotate dates
plt.show()

# Plotting Model Predictions vs Actual Prices
plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test, label='Actual Future Price', alpha=0.7)
plt.plot(y_test.index, predictions, label='Predicted Future Price', alpha=0.7, linestyle='--')
plt.title('Model Predictions vs Actual Prices')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
# Format the x-axis to show dates properly
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Auto-rotate dates
plt.show()


SyntaxError: invalid syntax (2587994824.py, line 75)